In [15]:
import json
from datetime import datetime
from pydantic import BaseModel, Field
from typing import Optional, List
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from langchain_core.exceptions import OutputParserException

# BookingRequest model
class BookingRequest(BaseModel):
    start_date: Optional[str] = Field(
        None,
        description="The starting date for the booking in format YYYY-MM-DD (e.g., 2025-05-12)."
    )
    start_time: Optional[str] = Field(
        None,
        description="The starting time for the booking in format HH:MM:SS AM/PM (e.g., 02:45:30 PM)."
    )
    duration_hours: Optional[float] = Field(
        None,
        description="The duration of the booking in hours (e.g., 0.5 for 30 minutes)."
    )
    capacity: Optional[int] = Field(
        None,
        description="The number of people the room should accommodate (e.g., 5)."
    )
    equipments: Optional[List[str]] = Field(
        default_factory=list,
        description="A list of equipment required (e.g., ['projector', 'whiteboard'])."
    )
    user_name: Optional[str] = Field(
        None,
        description="The name of the person making the booking."
    )
    clarification_needed: bool = Field(
        False,
        description="Indicates if clarification is required."
    )
    clarification_question: Optional[str] = Field(
        None,
        description="Question to ask for clarification if needed."
    )

# Examples
SUCCESS_EXAMPLE = {
    "start_date": "2025-07-16",
    "start_time": "10:00:00 PM",
    "duration_hours": 1,
    "capacity": 3,
    "equipments": ["whiteboard"],
    "user_name": "Heba",
    "clarification_needed": False,
    "clarification_question": None
}

MISSING_EXAMPLE = {
    "start_date": None,
    "start_time": "10:00:00 AM",
    "duration_hours": None,
    "capacity": 4,
    "equipments": [],
    "user_name": None,
    "clarification_needed": True,
    "clarification_question": "Could you provide the date and duration for the booking?"
}

# Prompt template
TEMPLATE = """
You are a booking assistant tasked with parsing a user's booking request into a structured JSON format based on the provided schema. Use the current date ({current_date}) and time ({current_time}) to interpret relative terms like "tomorrow" or "in 1 hour." If the input is ambiguous or lacks details, set `clarification_needed` to `true` and provide a `clarification_question`.

### User Request:
{user_request}

### Schema:
{parsing_schema}

### Instructions:
- For `start_date`, use YYYY-MM-DD format (e.g., "2025-05-12"). Derive from relative terms if provided.
- For `start_time`, use HH:MM:SS AM/PM format (e.g., "02:45:30 PM"). Derive from relative terms if provided.
- For `duration_hours`, use a float (e.g., 0.5 for 30 minutes).
- For `capacity`, use an integer (e.g., 5).
- For `equipments`, return an empty list (`[]`) if no equipment is specified. Do NOT use `null`.
- For optional fields, use `null` only if no default is specified and no value is provided.
- If the input is vague (e.g., "Hi"), set `clarification_needed` to `true` and provide a relevant `clarification_question`.

### Examples:
**Successful Example:**
{successful_example}

**Missing Example:**
{missing_example}

### Output:
Return a JSON object matching the schema. Ensure `equipments` is always a list, even if empty (`[]`).
"""

# Initialize LLM
def initialize_llm(name="groq"):
    if name == "groq":
        return ChatGroq(
            model_name="mixtral-8x7b-32768",
            api_key="YOUR_GROQ_API_KEY"  # Replace with your API key
        )
    raise ValueError(f"Unsupported LLM: {name}")

# Apply prompt template
def apply_prompt_template(parsing_schema: PydanticOutputParser) -> PromptTemplate:
    prompt_template = PromptTemplate(
        input_variables=["user_request", "current_date", "current_time"],
        template=TEMPLATE,
        partial_variables={
            "parsing_schema": parsing_schema.get_format_instructions(),
            "successful_example": json.dumps(SUCCESS_EXAMPLE, indent=2),
            "missing_example": json.dumps(MISSING_EXAMPLE, indent=2)
        }
    )
    return prompt_template

# State
state = {
    'user_input': "Hi",
    'llm_response': "",
    'messages': [],
    'parsed_request': {
        'start_date': None,
        'start_time': None,
        'duration_hours': None,
        'capacity': None,
        'equipments': [],
        'user_name': None
    },
    'clarification_needed': False,
    'clarification_question': None,
    'user_name_for_booking': None,
    'matching_rooms': [],
    'available_rooms': [],
    'alternative_rooms': [],
    'selected_room': None,
    'user_booking_confirmation_response': None,
    'booking_result': False,
    'error_message': None
}

# Main execution
current_date = datetime.now().strftime('%Y-%m-%d')  # e.g., 2025-05-13
current_time = datetime.now().strftime('%I:%M:%S %p')  # e.g., 10:27:00 PM
print(" >>>>> CURRENT DATE: %s" % current_date)
print(" >>>>> CURRENT TIME: %s" % current_time)

# Initialize parser and chain
parser = PydanticOutputParser(pydantic_object=BookingRequest)
prompt_template = apply_prompt_template(parser)
llm = initialize_llm(name="groq")
chain = prompt_template | llm | parser

# Update conversation history
print(" >>>>> USER INPUT : %s" % state['user_input'])
state["messages"].append(HumanMessage(content=state['user_input']))
conversation_context = "\n".join(
    f"{'USER' if isinstance(msg, HumanMessage) else 'AGENT'}: {msg.content}"
    for msg in state["messages"]
)
print(" >>>>> CONVERSATION CONTEXT: %s" % conversation_context)

# Invoke chain
try:
    parsed_data = chain.invoke({
        "user_request": conversation_context,
        "current_date": current_date,
        "current_time": current_time
    })
except OutputParserException as e:
    print(f"Parser Error: {e}")
    parsed_data = BookingRequest(
        start_date=None,
        start_time=None,
        duration_hours=None,
        capacity=None,
        equipments=[],
        user_name=None,
        clarification_needed=True,
        clarification_question="Could you provide more details about your booking request?"
    )

print(f" >>>>>>> CHAIN RESPONSE: {parsed_data}")
print(f" >>>>>>> PARSED REQUEST: {parsed_data.model_dump()}")
state.update({
    "parsed_request": parsed_data.model_dump(),
    "user_name_for_booking": parsed_data.user_name,
})

 >>>>> CURRENT DATE: 2025-05-13
 >>>>> CURRENT TIME: 10:32:17 PM


 >>>>> USER INPUT : Hi
 >>>>> CONVERSATION CONTEXT: USER: Hi


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [17]:
from pydantic import BaseModel, Field
from typing import Optional, List


class BookingRequest(BaseModel):
    start_date: Optional[str] = Field(
        None,
        description="The starting date for the booking in any format YYYY-MM-DD (e.g., 2025-05-12)" \
        "This can be a specific date or derived from relative terms like 'tomorrow' without asking for clarification again."
    )
    start_time: Optional[str] = Field(
        None,
        description="The starting time for the booking in the format HH:MM:SS AM/PM (e.g., 02:45:30 PM). " \
        "This can be a specific time or calculated from relative terms like 'after 1 hour' without asking for clarification again."
    )
    duration_hours: Optional[float] = Field(
        None,
        description="The duration of the booking in hours (e.g., 0.5 for 30 minutes, 1 for one hour)." \
        " Accepts fractional values for partial hours."
    )
    capacity: Optional[int] = Field(
        None,
        description="The number of people the room should accommodate (e.g., 5 for a room that fits 5 people)."
    )
    equipments: Optional[List[str]] = Field(
        default_factory=list,  # Default to an empty list if no value is provided
        # None,
        description="A list of equipment required for the booking (e.g., ['projector', 'whiteboard'])."
    )
    user_name: Optional[str] = Field(
        None,
        description="The name of the person making the booking to personalize the booking process."
    )
    clarification_needed: bool = Field(
        False,
        description="Indicates whether additional clarification is required to process the booking request"\
         " (e.g., True if the input is ambiguous or incomplete)."
    )
    clarification_question: Optional[str] = Field(
        None,
        description="A question to ask the user for clarification if required fields are nulls or ambiguous"\
             " (e.g., 'Could you specify the start time?')."
    )
    


In [18]:
%cd src

[WinError 2] The system cannot find the file specified: 'src'
d:\Heba\Hiring Task2\meeting_room_booking_agent\src


In [19]:
from datetime import datetime
from langchain.output_parsers import PydanticOutputParser

from langchain_core.messages import HumanMessage, SystemMessage

SUCCESS_EXAMPLE = {
    "start_date": "2025-07-16",
    "start_time": "10:00:00 PM", 
    "duration_hours": 1,
    "capacity": 3,
    "equipments": ["whiteboard"],
    "user_name": "Heba",
    "clarification_needed": False,
    "clarification_question": None
}

MISSING_EXAMPLE = {
    "start_date": None,  # Missing date
    "start_time": "10:00:00",  # Missing AM/PM, assumed AM
    "duration_hours": None,  # Missing duration
    "capacity": 4,
    "equipments": ["projector"],
    "user_name": None,
    "clarification_needed": True,
    "clarification_question": "Could you confirm if you meant 10:00:00 AM or PM, and provide the date for the booking?"
}



def apply_prompt_template(parsing_schema: PydanticOutputParser) -> PromptTemplate: 
    """
    Apply the prompt template to the LLM. Variables are defined in prompt_config.py
    """
    prompt_template = PromptTemplate(
        input_variables = ["user_request", "current_date", "current_time"],
        template = TEMPLATE,
        partial_variables = {
            "parsing_schema": parsing_schema.get_format_instructions(),
            "successful_example": json.dumps(SUCCESS_EXAMPLE, indent=2),
            "missing_example": json.dumps(MISSING_EXAMPLE, indent=2)
        })
    return prompt_template

def initialize_llm(name: str):
    """
    Initialize the LLM with tools. we can choose from different types of LLMs.
    """
    if name.lower() == "ollama":
        llm = ChatOllama(model_name=OLLAMA_MODEL_NAME,
                          ollama_api_key=OLLAMA_API_KEY,
                          temperature=0.0)
    
    elif name.lower() == "gemini":
        llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME,  # Explicitly pass the model
                                      google_api_key=GEMINI_API_KEY,
                                      temperature=0.0)
    elif name.lower() == "groq":
        llm = ChatGroq(model_name=GROQ_MODEL_NAME,
                        groq_api_key=GROQ_API_KEY,
                        temperature=0.0)
    else:
        raise ValueError(f"Unsupported LLM: {name}")
    
    llm.bind_tools([
        save_bookings_tool,
        check_time_conflict_tool,
        book_room_tool,
        find_matching_rooms_tool,
        find_similar_rooms_tool,
        find_rooms_by_equipments_tool
    ])

    return llm

In [20]:
state = {
    'user_input': "Hi",
    'llm_response': "",
    'messages': [],
    'parsed_request': {
        'start_date': None,
        'start_time': None,
        'duration_hours': None,
        'capacity': None,
        'equipments': [],
        'user_name': None
    },
    'clarification_needed': False,
    'clarification_question': None,
    'user_name_for_booking': None,

    'matching_rooms': [],
    'available_rooms': [],
    'alternative_rooms': [],
    'selected_room': None,
    'user_booking_confirmation_response': None,
    'booking_result': False,
    'error_message': None
}


############## (1.) Initialization ######################
current_date = datetime.now().strftime('%Y-%m-%d')    # e.g, 2025-05-12
current_time = datetime.now().strftime('%I:%M:%S %p') # e.g, 02:45:30 PM
print(" >>>>> CURRENT DATE: %s", current_date)
print(" >>>>> CURRENT TIME: %s", current_time)
# Initialize parser used for user request parsing
parser = PydanticOutputParser(pydantic_object=BookingRequest)
# Apply template to the inputrequest to inject the predefined template prompt
prompt_template = apply_prompt_template(parser)
# Initialize LLM
llm = initialize_llm(name="groq")
# Create chain
chain = prompt_template | llm | parser

############## (2.) Update conversation history ##################
print(" >>>>> USER INPUT : %s", state['user_input'])
state["messages"].append(HumanMessage(content=state['user_input']))
# Build full request context from conversation history ####
conversation_context = "\n".join(
    f"{'USER' if isinstance(msg, HumanMessage) else 'AGENT'}: {msg.content}"
    for msg in state["messages"]
)
print(" >>>>> CONVERSION CONTEXT: %s", conversation_context)

######################## (3.) Invoke chain ########################
parsed_data = chain.invoke({"user_request": conversation_context,
                            "current_date": current_date,
                            "current_time": current_time})
print(f" >>>>>>> CHAIN RESPONSE: {parsed_data}")
print(f" >>>>>>> PARSED REQUEST: {parsed_data.model_dump()}")
state.update({
    # "user_input": None,
    "parsed_request": parsed_data.model_dump(),
    "user_name_for_booking": parsed_data.user_name,
    })


 >>>>> CURRENT DATE: %s 2025-05-13
 >>>>> CURRENT TIME: %s 10:32:53 PM


 >>>>> USER INPUT : %s Hi
 >>>>> CONVERSION CONTEXT: %s USER: Hi
 >>>>>>> CHAIN RESPONSE: start_date=None start_time=None duration_hours=None capacity=None equipments=[] user_name=None clarification_needed=True clarification_question='Could you please provide more details about your booking request, such as the date, time, duration, capacity, and equipment required?'
 >>>>>>> PARSED REQUEST: {'start_date': None, 'start_time': None, 'duration_hours': None, 'capacity': None, 'equipments': [], 'user_name': None, 'clarification_needed': True, 'clarification_question': 'Could you please provide more details about your booking request, such as the date, time, duration, capacity, and equipment required?'}


---

In [3]:
llm = get_llm(name="groq")
request_parser = PydanticOutputParser(pydantic_object=ParseRequest)
prompt_template = apply_template(request_parser)

In [4]:
def parse_request(state: AgentState) -> AgentState:
    print("---NODE: PARSE REQUEST---")
    
    formatted_prompt = prompt_template.format(text=state["parsed_request"])
    response = llm.invoke(formatted_prompt).content
    response = request_parser.parse(response)
    state["parsed_request"] = response.model_dump()
    return state

In [ ]:
def ask_clarification(state: AgentState) -> AgentState:
    
    for field in state["missing_fields"]: 
        msgs = [MESSAGES[field] for field in state["missing_fields"]]
        
    state["clarification_question"] = "\n".join(msgs)
    state["clarification_needed"] = True
    return state

In [ ]:
def should_clarify(state: AgentState) -> AgentState:
    return "clarify_request_node" if state.get("clarification_needed") else "get_matching_rooms_node"


In [ ]:
from langgraph.graph import StateGraph, END

# Step 1: Define the graph
workflow = StateGraph(AgentState)

# Step 2: Add nodes
workflow.add_node("parse_request_node", parse_request)
workflow.add_node("clarify_request_node", lambda state: {"final_response": state.get("clarification_question")}) # Simple node to output question

# Step 3: Add conditional logic after parsing 
workflow.add_conditional_edges(
    "parse_request",
    is_valid_request,
    {
        True: END,
        False: "ask_clarification"
    }
)

workflow.add_conditional_edges(
    "ask_clarification",
    is_valid_request,
    {
        True: END,
        False: "ask_clarification"
    }
)

# Step 4: Add edge from clarification back to parsing
workflow.set_entry_point("parse_request")

# Step 5: Compile the graph
graph = workflow.compile()  

In [8]:
current_state: AgentState = {
    "original_request": "",
    "parsed_request": {},
    "is_valid_request": False,
    "clarification_needed": False,
    "clarification_question": None,
    "missing_fields": []
}

In [9]:
user_input = "I need a room for 4 people with a projector and whiteboard tomorrow at 10 AM for 1 hour."
current_state["original_request"] = user_input
output = graph.invoke(current_state)
print(output)

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`